Projeto feito para estudos sobre banco de dados e machine learning.

Dados retirados do desafio Kaggle: https://www.kaggle.com/competitions/titanic
Objetivo principal: Previsão sobre as mortes do Titanic.

# Dataset - Organização e Limpeza

In [168]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import model_selection
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [169]:
df_train = pd.read_csv('train.csv')
df_train.head(6)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q


In [170]:
df_test = pd.read_csv('test.csv')
df_test.head(6)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S


In [171]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Tratamento do dataset

### Dados ausentes


In [172]:
df_train.isnull().sum().sort_values(ascending=False)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Name             0
Pclass           0
Survived         0
Sex              0
Parch            0
SibSp            0
Fare             0
Ticket           0
dtype: int64

In [173]:
df_test.isnull().sum().sort_values(ascending=False)

Cabin          327
Age             86
Fare             1
Name             0
Pclass           0
PassengerId      0
Sex              0
Parch            0
SibSp            0
Ticket           0
Embarked         0
dtype: int64

In [174]:
#Tratamento dos 2 datasets.
data = [df_train, df_test]
#Age
for dataset in data:
    dataset['Age'] = dataset['Age'].fillna(dataset['Age'].mean())
#Fare
for dataset in data:
    dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].mean())


In [175]:
#Coluna Embarked

df_test['Embarked'].describe() #a moda é 'S', assim como o df_train

moda = 'S'

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna(moda)

df_train.isnull().sum().sort_values(ascending=False)

Cabin          687
PassengerId      0
Pclass           0
Survived         0
Name             0
Sex              0
SibSp            0
Age              0
Parch            0
Ticket           0
Fare             0
Embarked         0
dtype: int64

### Transformação dos dados

In [176]:
df_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


- É necessário transformar as colunas 'Sex' e 'Embarked' para numérico.

In [177]:
embarked_num = {
    'S':0,
    'C':1,
    'Q':2
}

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].replace(embarked_num)

genero = {
    'male': 0,
    'female': 1
}

for dataset in data:
    dataset['Sex'] = dataset['Sex'].replace(genero)

df_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0


In [178]:
#Tratamento dos títulos.

#Criar uma nova coluna com os títulos

for dataset in data:
    dataset['Title'] = dataset['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())


titles = {
    'Mr': 1,
    'Miss':2,
    'Mrs':3,
    'Master':4,
    'Rare':5
}

for dataset in data:
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    dataset['Title'] = dataset['Title'].replace(['Lady', 'The Countess', 'Don', 'Sir', 'Jonkheer', 'Capt', 'Col', 'Dr', 'Major', 'Rev', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].map(titles)
    dataset['Title'] = dataset['Title'].fillna(0)


df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",0,22.000000,1,0,A/5 21171,7.2500,NaN,0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.000000,1,0,PC 17599,71.2833,C85,1,3.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,0,2.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.000000,1,0,113803,53.1000,C123,0,3.0
4,5,0,3,"Allen, Mr. William Henry",0,35.000000,0,0,373450,8.0500,NaN,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,27.000000,0,0,211536,13.0000,NaN,0,5.0
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.000000,0,0,112053,30.0000,B42,0,2.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,29.699118,1,2,W./C. 6607,23.4500,NaN,0,2.0
889,890,1,1,"Behr, Mr. Karl Howell",0,26.000000,0,0,111369,30.0000,C148,1,1.0


- Retirar as colunas name e ticket

In [179]:
df_test = df_test.drop(columns=['Name', 'Cabin', 'Ticket'])
df_train= df_train.drop(columns=['Name', 'Cabin', 'Ticket'])

In [180]:
df_test

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,892,3,0,34.50000,0,0,7.8292,2,1
1,893,3,1,47.00000,1,0,7.0000,0,3
2,894,2,0,62.00000,0,0,9.6875,2,1
3,895,3,0,27.00000,0,0,8.6625,0,1
4,896,3,1,22.00000,1,1,12.2875,0,3
...,...,...,...,...,...,...,...,...,...
413,1305,3,0,30.27259,0,0,8.0500,0,1
414,1306,1,1,39.00000,0,0,108.9000,1,5
415,1307,3,0,38.50000,0,0,7.2500,0,1
416,1308,3,0,30.27259,0,0,8.0500,0,1


In [181]:
df_train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,0,22.000000,1,0,7.2500,0,1.0
1,2,1,1,1,38.000000,1,0,71.2833,1,3.0
2,3,1,3,1,26.000000,0,0,7.9250,0,2.0
3,4,1,1,1,35.000000,1,0,53.1000,0,3.0
4,5,0,3,0,35.000000,0,0,8.0500,0,1.0
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.000000,0,0,13.0000,0,5.0
887,888,1,1,1,19.000000,0,0,30.0000,0,2.0
888,889,0,3,1,29.699118,1,2,23.4500,0,2.0
889,890,1,1,0,26.000000,0,0,30.0000,1,1.0


MUDANÇAS A SE FAZER: 
- Tratar os outliers (pontos fora da curva)
- tamanho da famíliia: criar uma coluna familysize = SibSp + Parch + 1, e a coluna IsAlone (talvez 1 se é alone, 0 cc)
- Agrupamento por idades: criança, adulto e idosos.
- Tarifa: dividiir em quartis talvez (baixa, média, alta)

In [182]:
#Tamanho da família

for dataset in data:
    dataset['Family'] = dataset['SibSp'] + dataset['Parch'] + 1
    
for dataset in data:
    #quem viajava sozinho
    dataset['Alone'] = dataset['Family'].apply(lambda x: x if x == 1 else 0)



In [183]:
#Tratanto Outliers

def quartil():
    Q1 = df_train['Age'].quantile(q= 0.25)
    Q3 = df_train['Age'].quantile(q= 0.75)
    print(f'Q1 = {Q1} e Q3 = {Q3}')
    # Calculando a barreira externa.
    amp = Q3 - Q1
    limite_max = Q3 + 3 * amp
    limite_min = Q1 - 3 * amp
    print(f'Os Outliers da coluna Age estão entre {limite_min} e {limite_max}')
quartil() 

Q1 = 22.0 e Q3 = 35.0
Os Outliers da coluna Age estão entre -17.0 e 74.0


In [184]:
for dataset in data:
    dataset['Age'] = np.where(dataset['Age'] > 73, 73, dataset['Age'])

In [185]:
def quartil():
    Q1 = df_train['Fare'].quantile(q= 0.25)
    Q3 = df_train['Fare'].quantile(q= 0.75)
    print(f'Q1 = {Q1} e Q3 = {Q3}')
    # Calculando a barreira externa.
    amp = Q3 - Q1
    limite_max = Q3 + 3 * amp
    limite_min = Q1 - 3 * amp
    print(f'Os Outliers da coluna Fare estão entre {limite_min} e {limite_max}')
quartil() 

Q1 = 7.9104 e Q3 = 31.0
Os Outliers da coluna Fare estão entre -61.358399999999996 e 100.2688


In [186]:
for dataset in data:
    dataset['Fare'] = np.where(dataset['Fare']>100, 100, dataset['Fare'])

## Criação do Modelo

In [187]:
from sklearn.model_selection import train_test_split

In [188]:
#Separando a base de treino em X e Y. (Lembrete, estamos tentando criar uma função do tipo y = ax)
X = df_train.drop(['PassengerId', 'Survived'], axis = 1) #retirar a coluna survived pois ela será o y
y = df_train.Survived

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=7)


### Regressão Linear

In [189]:
from sklearn.linear_model import LinearRegression

In [190]:
modelo_regressao_linear=LinearRegression().fit(X_train, y_train)

In [191]:
y_pred_regressao_linear = modelo_regressao_linear.predict(X_test)

In [192]:
#Avaliação do Modelo
from sklearn.metrics import mean_squared_error, r2_score
print(f'Coeficiente a: {modelo_regressao_linear.coef_[0]}')
print(f'Coeficiente b (intercept): {modelo_regressao_linear.intercept_}')
print(f'MSE: {mean_squared_error(y_test, y_pred_regressao_linear)}')
print(f'R²: {r2_score(y_test, y_pred_regressao_linear)}')

Coeficiente a: -0.1679542014428703
Coeficiente b (intercept): 0.7282821221262523
MSE: 0.1618847595808244
R²: 0.32734810912331713


- Note que R² significa que o modelo não está convergindo aos dados reais. Será que outros modelos irão convergir melhor?

### Arvore de regressão

In [193]:
from sklearn import tree

In [194]:
modelo_arvore_regressao = tree.DecisionTreeRegressor(random_state=42).fit(X_train, y_train)

In [195]:
y_pred_arvoreReg=modelo_arvore_regressao.predict(X_test)

In [196]:
#Avaliação do modelo:
print(f'MSE: {mean_squared_error(y_test, y_pred_arvoreReg)}')
print(f'R²: {r2_score(y_test, y_pred_arvoreReg)}')

MSE: 0.2667372881355932
R²: -0.10832756398013754


- Note que este aparenta ser o pior modelo possível. R² < 0

### KNN

In [197]:
from sklearn.neighbors import KNeighborsRegressor

In [198]:
modelo_KNN = KNeighborsRegressor(n_neighbors=2).fit(X_train, y_train)

In [199]:
y_pred_KNN = modelo_KNN.predict(X_test)

In [200]:
#Avaliando modelo
print(f'R²: {r2_score(y_test, y_pred_KNN)}')
print(f'MSE: {mean_squared_error(y_test, y_pred_KNN)}')

R²: -0.15850601604278092
MSE: 0.2788135593220339


## Decision Tree

In [201]:
modelo_decision_tree = DecisionTreeClassifier(criterion='entropy', random_state= 7 )
modelo_decision_tree.fit(X_train, y_train)
previsao_Decision_Tree = modelo_decision_tree.predict(X_test)
result_treino = cross_val_score(modelo_decision_tree, df_train.drop('Survived', axis = 1), df_train['Survived'], cv = 5).mean()
print (f'taring score = {result_treino}')

taring score = 0.724053731718034


### Resultados

Até o momento, o melhor foi o método de Decision Tree. Este será submetido no site.

418, 295, 596, 596
